# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
from jsonpath import jsonpath
import datetime 
from datetime import timedelta ,time, datetime 
import re
from openpyxl import load_workbook
# from msoffcrypto_tool import OfficeFile
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean
import io
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [ ]:
# 黑名单
sql_black = '''
select cause, count(id_card) from db_credit.blacklist group by cause order by count(id_card) desc
'''
df_black = query(sql_black)
# df_black.to_excel('D:/需求/总需求/黑名单.xlsx', index=False)

In [ ]:
t_date = datetime.now().strftime('%Y%m%d')

近15天iPhone16全系列的机型转化数据

In [3]:
sql = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status, om.user_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id -- 去除json中的双引号
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
,om.buy_service_product,tso.status as service_status 
,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
, tor.update_time, tomt.reason, tprm.purchase_amount, tprm.liquidated_damages 违约金
, om.order_type, om.union_rent_tag, tojo.app_type, tpmn.name 机型
from  db_digua_business.t_order  om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id

-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 商品表
left join db_digua_business.t_product tp on tp.id = tod.product_id
-- 商品型号
left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 

where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
-- 京东外部订单关联表
left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id

where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and pa.type!=4
-- and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )                 -- 近15天数据
-- and  om.create_time < CURRENT_DATE
and DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-07-01'
'''
df_order = query(sql)
df_order.head()


,create_date,create_time,order_id,order_number,all_money,status,user_id,status2,channel_type,sku_attributes,product_name,new_actual_money,back_type,user_mobile,true_name,id_card_num,total_describes,decision_result,cancel_reason,trace_id,rejected,result,tips,is_vip,status_result,qvt_risk,qvt_result,channel_name,channel_type_id,activity_name,merchant_id,merchant_name,total_freeze_fund_amount,dy_order_item_json,type,buy_service_product,service_status,order_method,activity_id,appid,max_overdue_days,update_time,reason,purchase_amount,违约金,order_type,union_rent_tag,app_type,机型
0,2025-07-01,2025-07-01 00:00:33,3076812,A202507010000201,5899.57,10,22857200,已退款,其他渠道,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",首月1元 iPhone 16 Pro Max 非监管机 全新未激活 顺丰包邮,12898.7,租物归还,18368159875,汪小龙,341222200002198458,"[""系统自动操作审核订单A202507010000201，审核结果：不通过,拒绝理由：[\""...","{""code"":0,""grade"":""D"",""gradeMsg"":""评级风险较高，此类型订单...",None,ec818df5bbd2d97cd92d0a76305121f7,[命中冰鉴青云分强拒],【风控拒绝】风控不通过，租户风险较高。建议预收到设备价值或者拒绝。,[],None,1,None,None,未知渠道,4.0,京东活动,14,迪瓜优选,NaN,None,7.0,1,NaN,0,414,3F27D10A22175247FBDF164D4C74DA54,NaN,2025-07-01 00:01:06,系统风控拒绝转移,NaN,NaN,JD_ORDER,N,JR_APP,iPhone16ProMax
1,2025-07-01,2025-07-01 00:00:49,3076813,A202507010000492,8280.10,13,24804893,订单取消,芝麻租物,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",首月1元 iPhone16ProMax 全新未激活 非监管机 赠送壳膜 顺丰包邮,15596.3,租物归还,13178959899,林新仕,460006200601077215,None,"{""code"":0,""msg"":""ok"",""rejected"":[],""result"":""数...",订单超时,a5534ba0eca7abca76265a37c3c59c4c,[],数据校验通过,"[GPS最近地点:获取异常, 三地校验不一致:IP:海南万宁、定位:未知未知、收货地址:海南...",None,0,None,None,芝麻信用,2.0,芝麻租物-苹果16系列,14,迪瓜优选,NaN,None,2.0,1,NaN,1,323,2021003131606511,NaN,2025-07-01 00:00:52,订单初次分配,NaN,NaN,ZFB_ORDER,N,None,iPhone16ProMax
2,2025-07-01,2025-07-01 00:00:49,3076813,A202507010000492,8280.10,13,24804893,订单取消,芝麻租物,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",首月1元 iPhone16ProMax 全新未激活 非监管机 赠送壳膜 顺丰包邮,15596.3,租物归还,13178959899,林新仕,460006200601077215,None,"{""code"":0,""msg"":""ok"",""rejected"":[],""result"":""数...",订单超时,a5534ba0eca7abca76265a37c3c59c4c,[],数据校验通过,"[GPS最近地点:获取异常, 三地校验不一致:IP:海南万宁、定位:未知未知、收货地址:海南...",None,0,None,None,芝麻信用,2.0,芝麻租物-苹果16系列,14,迪瓜优选,NaN,None,2.0,1,NaN,1,323,2021003131606511,NaN,2025-07-01 00:00:52,None,NaN,NaN,ZFB_ORDER,N,None,iPhone16ProMax
3,2025-07-01,2025-07-01 00:01:28,3076814,A202507010001283,7798.10,10,24804894,已退款,芝麻租物,"[{""key"":""租赁方案"",""value"":""租完即送"",""type"":""text""},{...",iPhone 16 全新未激活 赠送壳膜 顺丰包邮,7798.1,租完即送,17873946162,刘博雄,430521200501042432,"[""发起修改套餐工单申请,由套餐:总租金：￥3296.5、买断价：￥7677.5租赁方案:租...","{""code"":0,""grade"":""D"",""gradeMsg"":""评级风险较高，此类型订单...","88888_胡春燕审核订单A202507010001283，审核结果：不通过,msg=客户申...",97a941ea5627ad12c8d46bb1e0f0e028,[],【风控复核】建议商户对租户进行电审，如: 1.通过核实用户的工作情况和累计贷款金额等，判断偿...,"[GPS最近地点:获取异常, IP与定位地址校验不一致:IP:湖南邵阳、定位:未知未知, 免...",0,0,1,1,芝麻信用,2.0,芝麻租物「严选」栏目-好商好品好服务好权益,14,维客自营账户,0.0,None,2.0,1,NaN,1,389,2021003131606511,NaN,2025-07-01 11:13:08,订单初次分配,NaN,NaN,ZFB_ORDER,N,None,iPhone16
4,2025-07-01,2025-07-01 00:02:23,3076815,A202507010002234,8280.10,13,24784259,订单取消,芝麻租物,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",首月1元 iPhone16ProMax 全新未激活 非监管机 赠送壳膜 顺丰包邮,15596.3,租物归还,13057300904,施亮亮,411528198609045872,None,"{""code"":0,""msg"":""ok"",""rejected"":[],""result"":""数...",信息填写错误,c29aa5ac2a3add8d777c9be759662dbf,[],数据校验通过,"[GPS最近地点:获取异常, IP与定位地址校验不一致:IP:江苏无锡、定位:未知未知]",None,0,None,None,芝麻信用,2.0,芝麻租物-苹果16系列,14,迪瓜优选,NaN,None,2.0,1,NaN,1,323,2021003131606511,NaN,2025-07-01 00:02:27,订单初次分配,NaN,NaN,ZFB_ORDER,N,None,iPhone16ProMax


In [4]:
df = df_order.copy()
df = df[df.type != 4]
df.order_id.notnull().sum()


np.int64(252700)

处理日期

In [5]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
df['年份'] = df["下单日期"].dt.year
df['hour'] = df['create_time'].dt.hour
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')

处理备注信息

In [6]:
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  


def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]

df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()


df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

颜色
白色钛金属     114410
黑色钛金属      53287
沙漠色钛金属     32187
原色钛金属      17317
白色         13628
           ...  
夜游黑            1
星云紫            1
极光             1
山茶白            1
银翼             1
Name: count, Length: 159, dtype: int64

内存
256GB          179487
512GB           42508
128GB           21121
12GB+256GB       1947
16GB+512GB       1506
12GB+512GB       1488
256G              917
128G              750
16GB+1TB          503
12GB+1TB          339
128GB WiFi版       118
256G WiFi版        109
512G              105
256GB WiFi版        62
16GB+256GB         30
64G                16
8GB+256GB          15
24GB+1TB            6
512GB WiFi版         6
512G WiFi版          5
256                 3
8GB+128GB           3
64G WiFi版           3
128G WiFi版          2
 256G               1
12GB+256G           1
12G+256GB           1
8G+256GB            1
Name: count, dtype: int64

进件
未进件    201679
进件      51021
Name: count, dtype: int64

# 订单去重

In [7]:
# 保留不去重的数据
df_no_drop = df.copy()

dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

定义状态

In [8]:
# note 2025-02-24添加
df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')
# , 'union_rent_tag', 'union_rent_rejected'
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
# note 2025-02-24修改
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)

NameError: name 'df_risk' is not defined

In [ ]:

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1) 
# df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截']),axis=1) 

保留商家的数据

In [9]:
df_contain = df.copy()

剔除商家数据

In [10]:
# 调用Data_Clean类中的drop_merchant函数
df = clean.drop_merchant(df)

去重订单明细导出

In [11]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-07-14'

导入服务订单

In [ ]:
# 筛选机型为iPhone16的订单
# df_16 = df[df['机型'].str.contains('iPhone16', na=False)]



In [ ]:
# # df_order = pd.read_excel('D:/需求/七月需求/订单明细.xlsx')
# df_order = pd.read_excel('D:/需求/七月需求/无标题(4).xlsx')
# # df_order = pd.read_excel('D:/需求/七月需求/无标题(3).xlsx')
# df_order.rename(columns={'order_number': '订单号', 'create_time': '下单时间', 'status':'订单状态', 'total_freeze_fund_amount': '冻结只有资金'}, inplace=True)
# df_order['下单日期']=df_order['下单时间'].dt.date
# df_order


In [ ]:
# df_order['全免押'] = np.where(df_order['冻结只有资金']==0,1,0)
# df_order['全不免押'] = np.where(df_order['冻结只有资金']==-1,1,0)
# df_order['部分免押'] = np.where((df_order['冻结只有资金']!=-1)&(df_order['冻结只有资金']!=0),1,0)
# df_order

In [ ]:
# df_kj = df_order[df_order['order_method']==1] # 选择订单方式为1的订单
# df_kj_group = df_kj.groupby('下单日期').agg({'订单号': 'count', '全免押': 'sum', '全不免押': 'sum', '部分免押':'sum'})
# # df_kj

In [ ]:
# df_fkj = df_order[df_order['order_method']==0]  # 选择订单方式为0的订单
# df_fkj_group = df_fkj.groupby('下单日期').agg({'订单号': 'count', '全免押': 'sum', '全不免押': 'sum', '部分免押':'sum'})

In [ ]:
# 筛选出所有与 "iPhone16" 相关的订单
# df_order['name'] = df_order['name'].fillna('未知')
# df_16 = df_order[df_order['name'].str.contains(pat='iPhone16', regex=True)]
# df_16_group = df_16.groupby('下单日期').agg({'订单号': 'count', '全免押': 'sum', '全不免押': 'sum', '部分免押':'sum'})

In [ ]:
# df_f16 = df_order[~df_order['name'].str.contains(pat='iPhone16', regex=True)]
# df_f16_group = df_f16.groupby('下单日期').agg({'订单号': 'count', '全免押': 'sum', '全不免押': 'sum', '部分免押':'sum'})

In [ ]:
# Today1 = datetime.now().strftime('%Y%m%d%H')
# with pd.ExcelWriter(f'D:/需求/七月需求/订单明细汇总_机型_{Today1}.xlsx', engine='openpyxl') as writer:
#     df_16_group.to_excel(writer, sheet_name='iPhone16')
#     df_f16_group.to_excel(writer, sheet_name='非iPhone16')

In [ ]:
# with pd.ExcelWriter(f'D:/需求/七月需求/订单明细汇总_快捷下单_{Today1}.xlsx', engine='openpyxl') as writer:
#     df_kj_group.to_excel(writer, sheet_name='快捷下单')
#     df_fkj_group.to_excel(writer, sheet_name='非快捷下单')

In [ ]:
# df_data = pd.read_excel('D:/日报/data.xlsx')
# df_data['日期'] = pd.to_datetime(df_data['日期'])
# df_data = df_data.set_index('日期')
# df_data = df_data[['去重订单', '前置拦截', '进件', '机审强拒', '电审拒绝', '客户取消', '无法联系', '出库前风控强拒']]

In [ ]:
# weekly_sum = W_group(custom_weekly_resampler(df_data)).drop(columns=['week_day'])
# group(weekly_sum)[['week_group', '去重订单','前置拦截','前置拦截率','进件','进件通过率','机审强拒','机审强拒比例','电审拒绝','电审拒绝比例','客户取消','客户取消率','无法联系','无法联系比例','出库前风控强拒','出库前风控强拒比例']]

In [ ]:
# df_ck3 = pd.read_excel('F:/租后表分析数据_ld/出库单三方数据表现-1114.xlsx')


In [ ]:
sql_3 = '''
SELECT om.order_number, tor.overdue_day
FROM db_digua_business.t_overdue_record tor 
left join db_digua_business.t_order om on tor.order_id=om.id
'''
df_3 = query(sql_3)

In [ ]:
# df_ck31 = df_ck3[['order_number', 'new_overdue_day']].drop_duplicates()
# df_31 = df_3.drop_duplicates()
# df_ck31
# df_ck3113 = pd.merge(df_31, df_ck31, on='order_number', how='right')
# df_ck3113[(df_ck3113['overdue_day'].notna())&(df_ck3113['new_overdue_day'].notna())&(df_ck3113['overdue_day']!=df_ck3113['new_overdue_day'])]

# 截止到当前时间首逾

## 查询订单

In [ ]:
# 截止当前时间的首逾逾期订单(已出库，以租后表为标准)的数量，逾期天数，应收租金new_actual_money，实收租金（已收）rent_received，未收租金new_actual_money-rent_received，分配人
sql = '''
select 
om.create_time, om.order_number, tprm.order_id, tprm.rent_received 实收租金
, ymos.refund_date, ymos.reality_refund_date
, om.status, tod.new_actual_money 应收租金
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
-- 资方订单分期表
left join (select order_id, refund_date, money, reality_refund_date from db_rent.ya_merchant_order_stages where sort=2) ymos on ymos.order_id=om.id
-- where date_format(om.create_time, '%Y-%m-%d')>='2025-05-12'
-- and date_format(om.create_time, '%Y-%m-%d')<='2025-06-12'  --设置时间范围
'''
df = query(sql)
df

,create_time,order_number,order_id,实收租金,refund_date,reality_refund_date,status,应收租金
0,2025-05-12 00:19:51,A2025051200195067,2620453,2401.58,2025-06-15,2025-06-11 09:13:16,4,15596.3
1,2025-05-12 03:35:25,A2025051203352480,2620961,1123.40,2025-06-15,2025-06-11 18:28:11,4,12996.7
2,2025-05-12 06:47:33,A2025051206473362,2621212,1335.00,2025-06-15,2025-06-11 09:13:21,4,12996.7
3,2025-05-12 07:57:09,A2025051207570893,2621325,1046.30,2025-06-15,2025-06-12 04:30:31,4,12996.7
4,2025-05-12 08:02:39,A202505120802384,2621330,969.20,2025-06-15,2025-06-11 09:13:34,4,12996.7
...,...,...,...,...,...,...,...,...
5394,2025-06-11 14:32:10,A20250611143210435,2875447,991.12,2025-07-14,2025-07-12 17:48:30,4,12996.7
5395,2025-06-12 16:54:09,A20250612165409931,2887736,249.00,NaT,NaT,8,7182.0
5396,2025-06-11 14:07:33,A2025061114072394,2875109,1049.00,2025-07-14,2025-06-30 17:20:45,4,11608.7
5397,2025-06-12 17:29:22,A20250612172921436,2888249,668.68,2025-07-15,NaT,4,12996.7


## 逾期天数

In [25]:
# 截止当前时间的首逾逾期订单(已出库，以租后表为标准)的数量，逾期天数，应收租金，实收租金，未收租金，分配人

t_date = datetime.now().date()
df.loc[:, 'now_day'] = pd.to_datetime(datetime.now().date())
df.loc[:, '下单日期'] = pd.to_datetime(df.create_time.dt.date)
df.loc[:, 'reality_refund_date'] = df.reality_refund_date.dt.date
# 首逾：sort=2，refund_date<now_day|reality_refund_date>refund_date
df.loc[:, 'overdue_day'] = np.where(df.reality_refund_date.isna(), np.where(df.refund_date<df.now_day, df.now_day-df.refund_date, 0), 
                                    np.where(df.reality_refund_date>df.refund_date, df.reality_refund_date-df.refund_date, 0))
df.loc[:, 'overdue_day'] = df.overdue_day.apply(lambda x: x.days)
df.loc[:, '未收租金'] = df.应收租金-df.实收租金
df_new = df[(df.overdue_day>0)&(df.reality_refund_date.isna())&(df.status==4)]
df_new

,create_time,order_number,order_id,实收租金,refund_date,reality_refund_date,status,应收租金,now_day,下单日期,overdue_day,未收租金
55,2025-05-12 11:51:59,A20250512115159694,2623104,1301.00,2025-06-15,NaT,4,7798.1,2025-07-15,2025-05-12,30,6497.10
71,2025-05-12 12:49:03,A20250512124902536,2623756,468.40,2025-06-15,NaT,4,9097.3,2025-07-15,2025-05-12,30,8628.90
133,2025-05-12 16:03:00,A2025051216030045,2626274,533.07,2025-06-15,NaT,4,12996.7,2025-07-15,2025-05-12,30,12463.63
156,2025-05-12 16:54:44,A20250512165444876,2627105,2168.30,2025-06-15,NaT,4,12997.7,2025-07-15,2025-05-12,30,10829.40
191,2025-05-12 19:18:05,A20250512191804162,2628640,562.91,2025-06-15,NaT,4,12996.7,2025-07-15,2025-05-12,30,12433.79
...,...,...,...,...,...,...,...,...,...,...,...,...
5129,2025-06-11 17:51:47,A20250611175147669,2878487,668.68,2025-07-14,NaT,4,12996.7,2025-07-15,2025-06-11,1,12328.02
5137,2025-06-11 18:23:19,A20250611182318281,2878853,510.41,2025-07-14,NaT,4,12996.7,2025-07-15,2025-06-11,1,12486.29
5164,2025-06-11 16:11:06,A20250611161105176,2877122,292.75,2025-07-14,NaT,4,7797.5,2025-07-15,2025-06-11,1,7504.75
5177,2025-06-11 18:51:41,A20250611185140578,2879147,1507.60,2025-07-14,NaT,4,15596.3,2025-07-15,2025-06-11,1,14088.70


## 分配人

In [26]:
sql_name = '''
        SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
        FROM db_digua_business.t_user_verify_order_record tuvor
        left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
        where tuvor.del_flag = 0 ORDER BY tuvor.update_time
        '''
df_name = query(sql_name)
df_name


,order_id,分配人,update_time
0,703280,李巧凤,2023-10-16 16:02:04
1,703287,小谢,2023-10-16 16:02:04
2,703288,小张,2023-10-16 16:02:04
3,699105,李巧凤,2023-10-16 16:09:36
4,699143,李巧凤,2023-10-16 16:09:36
...,...,...,...
216749,3251761,杨健,2025-07-15 18:36:08
216750,3251765,胡彩滢,2025-07-15 18:36:10
216751,3251737,小平,2025-07-15 18:36:12
216752,3251707,贤鹏,2025-07-15 18:36:14


In [28]:
df_name = df_name.sort_values(by='update_time', ascending=False).groupby('order_id').head(1)
# 创建分配人映射关系 
name_dict = {
        '小张': '李巧玲',
        '小周': '李巧凤',
        '小南': '刘三妹',
        '小何': '何静',
        '小谢': '谢金凤',
        '小咪': '杨健',
        '小慧': '林思慧',
        '小滢': '胡彩滢',
        '小星': '廖丽敏',
        '小兰': '黄兰娟',
        '小晚': '周莹',
        '小芳': '罗芳'
        }
name_list = ['李巧玲','李巧凤','刘三妹','何静','谢金凤','廖丽敏','李楠','黄兰娟','杨健','林思慧','胡彩滢','周莹','罗芳', '周念慈', '周汉鸿']
# 匹配映射字典
df_name.loc[:, '信审员'] = df_name.分配人.apply(lambda x: name_dict[x] if str(x).startswith('小') and str(x) in name_dict.keys() else x)
# df_name = df_name[df_name.信审员.isin(name_list)]
df_name

,order_id,分配人,update_time,信审员
216753,3251751,文静,2025-07-15 18:36:15,文静
216752,3251707,贤鹏,2025-07-15 18:36:14,贤鹏
216751,3251737,小平,2025-07-15 18:36:12,小平
216750,3251765,胡彩滢,2025-07-15 18:36:10,胡彩滢
216749,3251761,杨健,2025-07-15 18:36:08,杨健
...,...,...,...,...
37,702113,李巧凤,2023-10-16 16:09:36,李巧凤
36,702045,李巧凤,2023-10-16 16:09:36,李巧凤
2,703288,小张,2023-10-16 16:02:04,李巧玲
1,703287,小谢,2023-10-16 16:02:04,谢金凤


In [29]:
# 关联分配人
df_all = df_new.merge(df_name[['order_id', '信审员']], on='order_id', how='left')
df_all

,create_time,order_number,order_id,实收租金,refund_date,reality_refund_date,status,应收租金,now_day,下单日期,overdue_day,未收租金,信审员
0,2025-05-12 11:51:59,A20250512115159694,2623104,1301.00,2025-06-15,NaT,4,7798.1,2025-07-15,2025-05-12,30,6497.10,钟桂洪
1,2025-05-12 12:49:03,A20250512124902536,2623756,468.40,2025-06-15,NaT,4,9097.3,2025-07-15,2025-05-12,30,8628.90,黄兰娟
2,2025-05-12 16:03:00,A2025051216030045,2626274,533.07,2025-06-15,NaT,4,12996.7,2025-07-15,2025-05-12,30,12463.63,林婉婷
3,2025-05-12 16:54:44,A20250512165444876,2627105,2168.30,2025-06-15,NaT,4,12997.7,2025-07-15,2025-05-12,30,10829.40,胡春燕
4,2025-05-12 19:18:05,A20250512191804162,2628640,562.91,2025-06-15,NaT,4,12996.7,2025-07-15,2025-05-12,30,12433.79,林婉婷
...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,2025-06-11 17:51:47,A20250611175147669,2878487,668.68,2025-07-14,NaT,4,12996.7,2025-07-15,2025-06-11,1,12328.02,廖丽敏
227,2025-06-11 18:23:19,A20250611182318281,2878853,510.41,2025-07-14,NaT,4,12996.7,2025-07-15,2025-06-11,1,12486.29,罗芳
228,2025-06-11 16:11:06,A20250611161105176,2877122,292.75,2025-07-14,NaT,4,7797.5,2025-07-15,2025-06-11,1,7504.75,刘三妹
229,2025-06-11 18:51:41,A20250611185140578,2879147,1507.60,2025-07-14,NaT,4,15596.3,2025-07-15,2025-06-11,1,14088.70,何静


# 六月份买断

## 查询订单

In [10]:
# 统计六月份买断（买断表t_order_out，买断时间pay_date筛选）的订单数、使用了多少买断优惠劵、优惠前买断金额、优惠后买断金额、采购金额、毛利率（优惠前/后买断金额-采购金额）/采购金额
# ybt.`status`'任务状态，0为待处理，1为跟进中，2已挂起，3已完成，4已取消'
sql_md= f'''
select
too.order_id 订单ID, tod.new_actual_money 优惠前买断金额, too.discount_money 优惠金额, (tod.new_actual_money-too.discount_money) 优惠后买断金额,
tprm.purchase_amount 采购金额, om.has_actual, date_format(too.pay_date, '%Y-%m') 月份
from db_digua_business.t_order_out too
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=too.order_id
left join db_digua_business.t_order_details tod on tod.order_id=too.order_id
left join db_digua_business.t_order om on om.id=too.order_id
where date_format(too.pay_date, '%Y-%m')='2025-06'
and too.status=2
'''
df_md = query(sql_md)
df_md

,订单ID,优惠前买断金额,优惠金额,优惠后买断金额,采购金额,has_actual,月份
0,1205846,5197.90,0.0,5197.90,3649.0,1,2025-06
1,850763,5197.57,0.0,5197.57,2250.0,1,2025-06
2,1500435,12416.80,0.0,12416.80,8903.0,1,2025-06
3,1349599,12996.70,0.0,12996.70,8108.0,1,2025-06
4,1182116,12996.70,0.0,12996.70,7970.0,1,2025-06
...,...,...,...,...,...,...,...
697,497019,14135.96,0.0,14135.96,8280.0,1,2025-06
698,1055232,10397.10,0.0,10397.10,6898.0,1,2025-06
699,932474,9097.30,0.0,9097.30,5867.0,1,2025-06
700,1401884,12996.70,0.0,12996.70,8108.0,1,2025-06


In [15]:

df_md.loc[:, '实际买断订单数'] = np.where(df_md.has_actual == 1, 1, 0)
# df_md.loc[:, '使用优惠劵订单数'] = np.where((df_md.status.isin([3, 4])) & (df_md.has_actual == 1) & (df_md.discount_money > 0), 1, 0)
df_md.loc[:, '使用优惠劵订单数'] = np.where(df_md.优惠金额 > 0, 1, 0)
# df_md.loc[:, '优惠后买断金额'] = np.where((df_md.status.isin([3, 4])) & (df_md.has_actual == 1),df_md.优惠后买断金额, 0)
df_md.groupby('月份').agg({'订单ID': 'count','使用优惠劵订单数': 'sum', '优惠前买断金额': 'sum', '优惠后买断金额': 'sum', '采购金额': 'sum'}).rename(columns={'订单ID': '买断订单数', '优惠金额': '使用优惠劵订单数'})
df_md.loc[:, '发放优惠前的毛利率'] = ((df_md.优惠前买断金额 - df_md.采购金额) / df_md.采购金额).map(
        lambda x: format(x, '.2%'))
df_md.loc[:, '发放优惠后的毛利率'] = ((df_md.优惠后买断金额 - df_md.采购金额) / df_md.采购金额).map(
    lambda x: format(x, '.2%'))
# df_md
    

,买断订单数,使用优惠劵订单数,优惠前买断金额,优惠后买断金额,采购金额
月份,,,,,
2025-06,702,147,7937068.24,7844411.4,4862847.0


# 查询手机号对应的订单

In [6]:
# 查询excel文件，sheet2页面中的手机号列，与数据库订单表中7月份订单手机号列进行匹配，并返回匹配结果的订单号和手机号
df_phone_number = pd.read_excel('D:/需求/进件记录.xlsx', sheet_name='Sheet2')
# 转str类型
df_phone_number['手机号'] = df_phone_number['手机号'].astype(str)
# 取出第一列数据去重，并转换为列表
# phone_numbers = df_phone_number.iloc[:, 0].drop_duplicates().tolist()
# phone_numbers
phone_number_list = tuple(df_phone_number.iloc[:, 0].drop_duplicates().to_list())
phone_number_list


('17313800612',
 '13382034154',
 '17621364999',
 '13896381287',
 '15760956918',
 '18736356722',
 '18956997590',
 '15343950336',
 '18271819573',
 '13706659205',
 '15196652755',
 '15215709087',
 '17585251042',
 '18511287510',
 '18523624276',
 '13120010414',
 '18600018215',
 '15606199334',
 '17521521745',
 '18565684777',
 '13262551290',
 '18559980115',
 '13091596158',
 '15559806821',
 '18241421310',
 '15611443666',
 '15606095041',
 '17502839106',
 '15606555267',
 '17612195051',
 '13088988906',
 '13215631226',
 '18561392934',
 '15527540026',
 '13242802314',
 '18222745822',
 '13072973000',
 '18650737681',
 '13066727210',
 '15360837760',
 '15524224933',
 '13802768134',
 '19561351980',
 '13605619401',
 '18594207654',
 '13875153629',
 '18487572077',
 '19086801672',
 '15244559922',
 '15231383222',
 '18212366375',
 '19564903147',
 '13477271012',
 '13277830591',
 '15125751483',
 '18922424606',
 '15728317926',
 '17733966803',
 '13891699578',
 '18724190611',
 '18266758505',
 '13599752758',
 '136242

In [33]:
df.user_mobile.dtype

dtype('O')

In [7]:
df = df[df.user_mobile.str.contains(r'\d+')]
# 在df中筛选出手机号在phone_numbers列表中的行
df_filtered = df[df['user_mobile'].isin(phone_numbers)]
# 返回匹配结果的订单号和手机号
df_ans = df_filtered[['order_number', 'user_mobile']]
df_ans



NameError: name 'df' is not defined

In [8]:


# 查询订单
sql = f'''
select order_number,user_mobile from db_digua_business.t_order where create_time between '2023-07-01' and '2023-07-15' and user_mobile in {phone_number_list}
'''
df = query(sql)
df

,order_number,user_mobile
0,A2023070701365956,17178519371
1,A2023071417350348,13639481600


# 二手汇总

In [ ]:
excel_file = pd.ExcelFile('F:/需求/七月需求/24年1月-10月机型分析.xlsx')
df = excel_file.parse('1月')[:-1]
for sheet in excel_file.sheet_names[1:]:
    df = pd.concat([df,excel_file.parse(sheet)[:-1]])
df

In [ ]:
# 太过久远，字段已无用
# df['型号'] = df['型号'].str.replace(' ', '', regex=False)
# df
# df_group = df.groupby('型号').agg({'数量': 'sum', '总回收价格':'sum', '总采购金额': 'sum', '总官网金额': 'sum'})
# df_group['总回收价格均值'] = df_group['总回收价格']/df_group['数量']
# df_group['总采购金额均值'] = df_group['总采购金额']/df_group['数量']
# df_group['总官网金额均值'] = df_group['总官网金额']/df_group['数量']
# df_group = df_group.reset_index()
# df_group['型号_new'] = df_group['型号'].str.split('/').str[0]
# df_group['内存'] = df_group['型号'].str.split('/').str[1]
# df_group.sort_values(['数量'], ascending=[False]).to_excel('F:/需求/七月需求/24年1月-10月机型分析汇总.xlsx')

# 惜仪需求

In [ ]:
df_es = pd.read_excel('F:/需求/二手手机/二手机型.xlsx')

# 字段可能需要修改
df_es['月份'] = df_es['入库日期'].dt.month
df_es['型号内存'] = df_es['型号_new'] + '_' + df_es['内存'] + '_' + df_es['颜色']

In [ ]:
month_list = df_es['月份'].drop_duplicates().to_list()
for ml in month_list:
    df_month = df_es[df_es['月份']==ml]
    df_group = df_month.groupby('型号内存').agg({'颜色': 'count', '回收价格': 'sum', '采购价':'sum', '官网价':'sum'})
    df_group.rename(columns={'颜色': '数量', '回收价格': '总回收价格', '采购价': '总采购价', '官网价': '总官网价'}, inplace=True)
    df_group.loc['合计'] = {'数量': df_group['数量'].sum(), '总回收价格': df_group['总回收价格'].sum(), '总采购价': df_group['总采购价'].sum(),
                                        '总官网价': df_group['总官网价'].sum()}
    df_group['总采购价比值'] = df_group['总回收价格']/df_group['总采购价']
    df_group['总采购价比值'] = df_group['总采购价比值'].apply(lambda x: format(x, '.2%'))

    df_group['总官网价比值'] = df_group['总回收价格']/df_group['总官网价']
    df_group['总官网价比值'] = df_group['总官网价比值'].apply(lambda x: format(x, '.2%'))
    # if ml==1:
    #     with pd.ExcelWriter('F:/需求/二手手机/二手机型（月份）.xlsx', engine='openpyxl') as writer:
    #         df_group.to_excel(writer, sheet_name=f'{ml}月')
    # else:
    #     with pd.ExcelWriter('F:/需求/二手手机/二手机型（月份）.xlsx', engine='openpyxl', mode='a') as writer:
    #         df_group.to_excel(writer, sheet_name=f'{ml}月')

In [ ]:
sheets = pd.ExcelFile('F:/需求/二手手机/二手机型（月份）.xlsx')
df = sheets.parse('9月')[:-1]
for sheet in sheets.sheet_names[9:]:
    df = pd.concat([df,sheets.parse(sheet)[:-1]])
df


In [ ]:
def model_price(df):
    df_group = df.groupby('型号内存').agg({'数量': 'sum', '总回收价格':'sum', '总采购价': 'sum', '总官网价': 'sum'})
    df_group['总回收价格均值'] = df_group['总回收价格']/df_group['数量']
    df_group['总采购价均值'] = df_group['总采购价']/df_group['数量']
    df_group['总官网价均值'] = df_group['总官网价']/df_group['数量']
    df_group = df_group.reset_index()
    df_group['型号'] = df_group['型号内存'].str.split('_').str[0]
    df_group['内存'] = df_group['型号内存'].str.split('_').str[1]
    df_group['颜色'] = df_group['型号内存'].str.split('_').str[2]
    return df_group

In [ ]:
df['型号内存'] = df['型号内存'].str.replace(' ', '', regex=False)
df = df[df['型号内存'].str.contains((r'iPhone14|iPhone15'))]
df = df[df['型号内存'].str.contains((r'99新|95新|准新|90新'))]
df_ds = df[df['型号内存'].str.contains((r'带锁|有锁'))]
df_wds = df[~df['型号内存'].str.contains((r'带锁|有锁'))]
# model_price(df_ds).to_excel(f'F:/需求/七月需求/24年近3个月机型分析汇总_{t_date}_带锁.xlsx')
# model_price(df_wds).to_excel(f'F:/需求/七月需求/24年近3个月机型分析汇总_{t_date}_未带锁.xlsx')
# df_group.sort_values(['数量'], ascending=[False]).to_excel(f'F:/需求/七月需求/24年1月-10月机型分析汇总_{t_date}.xlsx')
# df_group.sort_values(['数量'], ascending=[False])#.to_excel(f'F:/需求/七月需求/24年近3个月机型分析汇总_{t_date}.xlsx')

# 获取优惠5折数据

In [ ]:
df_5 = pd.read_excel('F:/需求/瑞彤需求/总赠送5折优惠卷.xlsx')
df_5['订单号'] = df_5['订单号'].apply(lambda x: x.split('\\')[0].strip())
order_list = tuple(df_5['订单号'].to_list())
df_5['订单号'].shape

In [ ]:
sql=f'''
SELECT tprm.order_number 订单号, tprm.contract_price 买断价, tprm.purchase_amount 采购价,  tos.money 月租金, (tprm.contract_price-tprm.all_rental) 买断尾款
, tprm.all_deposit 商品押金, tprm.all_rental 总租金, tod.product_name 产品名称
FROM db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order_stages tos on tos.order_id=tprm.order_id
left join db_digua_business.t_order_details tod on tod.order_id=tprm.order_id
WHERE tprm.order_number in {order_list} and tos.sort=2
'''
df = query(sql)

In [ ]:
# df.to_excel('F:/需求/瑞彤需求/总赠送5折优惠卷_new.xlsx', index=False)

In [ ]:
df_es = pd.read_excel('F:/需求/二手手机/22年-23年二手机型.xlsx')
df_es = df_es[df_es['订单号'].str.contains('A2023')]


In [ ]:
df_es_group = df_es.groupby(['型号', '内存']).agg({'订单号': 'count', '售价': 'sum', '采购成本': 'sum', '官网价': 'sum'})
df_es_group['回收价'] = df_es_group['售价'] / df_es_group['订单号']
df_es_group['采购价'] = df_es_group['采购成本'] / df_es_group['订单号']
df_es_group['官网价均值'] = df_es_group['官网价'] / df_es_group['订单号']
# df_es_group

# 商家

In [ ]:
# 黑名单
sql4 = '''
SELECT
id_card, mobile
FROM db_credit.blacklist b
WHERE b.end_time is NULL or b.end_time>SYSDATE() -- 结束时间为空且大于当前时间，则为黑名单
'''
df4 = query(sql4)

In [ ]:
# 处理mobile字段并修改字段类型
df4 = df4[df4.mobile.str.contains(r'\d+')]
df4['id_card'] = df4['id_card'].astype(str)
df4['mobile'] = df4['mobile'].astype(int)
df4

In [ ]:
# 导入Excel文件
df_customer = pd.ExcelFile('F:/需求/总需求/商家/客户名单.xlsx')
df_customer1 = df_customer.parse(sheet_name='三要素')
df_customer2 = df_customer.parse(sheet_name='二要素')

In [ ]:
df_customer1['now_date'] = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))
df_customer1

In [ ]:
# 三要素 匹配身份证号和手机号
df_customer1['身份证'] = df_customer1['身份证'].astype(str)
df_customer_list = df_customer1.身份证.to_list()
df4[df4.id_card.isin(df_customer_list)]
# 匹配手机号
df_customer_mobile_list = df_customer1.手机号.to_list()
df4[df4.mobile.isin(df_customer_mobile_list)]

In [ ]:
# 匹配手机号
df_customer_mobile_list2 = df_customer2.mobile.to_list()
df4[df4.mobile.isin(df_customer_mobile_list2)]

## 年龄分布

In [ ]:
# 定义一个函数来计算年龄
def calculate_age(id_card, order_time):
    birth_year = int(id_card[6:10])
    birth_month = int(id_card[10:12])
    birth_day = int(id_card[12:14])
    birth_date = datetime(birth_year, birth_month, birth_day)
    age = order_time.year - birth_date.year - ((order_time.month, order_time.day) < (birth_date.month, birth_date.day))
    return age

# 应用函数到 DataFrame
df_customer1['age'] = df_customer1.apply(lambda row: calculate_age(row['身份证'], row['now_date']), axis=1)


In [ ]:
# 定义年龄区间
age_bins = [-999, 18, 20, 23, 25, 30, 35, 40, 45, float('inf')]
age_labels = ['[-999,18)','[18,20)',  '[20,23)', '[23,25)', '[25,30)', '[30,35)', '[35,40)', '[40,45)', '[45+)']

# 使用 pd.cut 设置年龄区间
df_customer1['age_group'] = pd.cut(df_customer1['age'], bins=age_bins, labels=age_labels, right=True, include_lowest=False)


In [ ]:
df_customer1[df_customer1.age<=26].shape

In [ ]:
# 三要素
df_age = df_customer1.groupby('age_group').agg(数量=('身份证', 'count'))
df_age.to_excel('F:/需求/总需求/商家/年龄分布1.xlsx')

# 交易流水汇总

In [ ]:
df_ls = pd.read_excel('F:/需求/七月需求/无标题(14).xlsx')

In [ ]:
df_ls.loc[:, '交易发起日期'] = df_ls.交易发起时间.dt.date

In [ ]:
# df_ls.groupby('交易发起日期').agg(总应付金额=('应付金额', 'sum'), 总实付金额=('实付金额（如果为空则应付=实付）', 'sum')).to_excel('F:/需求/七月需求/交易流水汇总.xlsx')

In [ ]:
df_ls2 = pd.read_excel('F:/需求/七月需求/无标题(15).xlsx')

In [ ]:
def pay_way(way):
    if way == 'WX':
        return '微信'
    elif way == 'WXXCX':
        return  "微信小程序"
    elif way == 'ZFB':
        return '支付宝'
    elif way == 'ZFBYSQ':
        return '支付宝预授权'
    elif way == 'ZFBXCX':
        return '支付宝小程序'
    elif way == 'DYXCX':
        return '抖音小程序'
    elif way == 'DY_HX_PAY':
        return '抖音小程序核销'
    elif way == 'YE':
        return '余额'
    elif way == 'WXZFF':
        return '微信支付分'
    elif way == 'ZFBZQQ':
        return '支付宝周期扣款'
    elif way == 'XX':
        return '线下'
    elif way == 'UNIONXX':
        return '银联线下'
    elif way == 'DEPOSIT':
        return '押金抵扣'
    elif way == 'RY_PAY':
        return '第三方代扣'
    elif way == 'ANT_PAY':
        return '蚂蚁链代扣'
    elif way == 'ZYX_PAY':
        return '智赢行支付'
    elif way == 'ZFB_WAP_PAY':
        return '支付宝h5'
    elif way == 'ZFBYWD':
        return '支付宝业务单代扣'
    elif way == 'ZFXXSK':
        return '线下（对资方）支付'
    elif way == 'XXX':
        return '线下（对维客）支付'
    elif way == 'XX_MERCHANT':
        return '线下（对商家）支付'
    elif way == 'UNICOM_PAY':
        return '联通号卡支付'

In [ ]:
df_ls2.loc[:, '支付方式_中文'] = df_ls2.apply(lambda x: pay_way(x['支付方式']), axis=1)
# df_ls2.to_excel('F:/需求/七月需求/支付宝交易流水.xlsx')

In [ ]:
df_ls2.loc[:, '交易发起日期'] = df_ls2.交易发起时间.dt.date
df_ls2_3 = df_ls2[df_ls2.支付方式_中文.isin(['支付宝小程序', '支付宝业务单代扣', '支付宝预授权'])]
df_ls2_3_group = df_ls2_3.groupby('交易发起日期').agg(总应付金额_前三=('应付', 'sum'), 总实付金额_前三=('实付', 'sum'))
df_ls2_4 = df_ls2[df_ls2.支付方式_中文.isin(['支付宝小程序', '支付宝业务单代扣', '支付宝预授权', '智赢行支付'])]
df_ls2_4_group = df_ls2_4.groupby('交易发起日期').agg(总应付金额_后四=('应付', 'sum'), 总实付金额_后四=('实付', 'sum'))
df_ls2_group = df_ls2_3_group.merge(df_ls2_4_group, on='交易发起日期', how='inner').to_excel('F:/需求/七月需求/支付方式交易汇总.xlsx')

# 匹配云商分

In [ ]:
# trace关联
sql_trace='''-- trace关联表 
SELECT order_trace_id, trace_id, order_id, order_number, risk_trace_id
from db_credit.order_association
;
'''
trace_df = query(sql_trace)
# 云商分
sql1='''-- 风控-百融贷前-云商分
-- 101331	101331	101331
SELECT trace_id, query_rs
from db_credit.risk_bairong_scoreysstd
;
'''
br_ys_df =query(sql1)
br_ys_df['云商分'] = br_ys_df['query_rs'].apply(lambda x: json.loads(x).get('Score', {}).get('scoreysstd', None))
br_ys_df['云商分'] = br_ys_df['云商分'].astype(int)
df_rule = trace_df.merge(br_ys_df[['trace_id', '云商分']], on='trace_id', how='outer')


In [ ]:
df_y = pd.read_excel('F:/租后表分析数据_ld/tmp(1).xlsx')

In [ ]:
# df_y[['order_id', 'order_number', '拒绝理由']].merge(df_rule[['order_id', '云商分']], on='order_id', how='left').to_excel('F:/需求/总需求/强拒云商分.xlsx', index=False)

In [ ]:
# （26）年龄、性别、总租金
# 定义一个函数来计算年龄
def calculate_age1(id_card, order_time):
    birth_year = int(id_card[6:10])
    birth_month = int(id_card[10:12])
    birth_day = int(id_card[12:14])
    birth_date = datetime(birth_year, birth_month, birth_day)
    age = order_time.year - birth_date.year - ((order_time.month, order_time.day) < (birth_date.month, birth_date.day))
    return age

# 定义一个函数来判断性别
def get_gender(id_card):
    gender_digit = int(id_card[-2])  # 第17位数字
    return '男' if gender_digit % 2 != 0 else '女'

# 取基本信息
sql = '''
SELECT  om.order_number, om.create_time
,om.all_money as all_rental
,tmu.id_card_num
,tod.actual_money

from db_digua_business.t_order om
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on om.id = tod.order_id
-- where date_format(om.create_time, '%Y-%m-%d') >= '2024-11-01'
;
'''

df_zhys = query(sql)

# df_zhys.dtypes
df_zhys['下单日期'] = df_zhys['create_time'].dt.date
df_zhys['order_number'] = df_zhys['order_number'].astype(str)
# 取样本
path = 'F:/租后表分析数据_ld/'
data_model = pd.read_excel(path + 'tmp(2).xlsx')
data_model['order_number'] = data_model['order_number'].astype(str)
# data_model


In [ ]:
# 关联
data_model = data_model.merge(df_zhys, on = 'order_number', how = 'left')
data_model.shape
data_model[:2]

In [ ]:

data_model1 = data_model[data_model.id_card_num.isna()]
data_model2 = data_model[data_model.id_card_num.notna()]
# data_model = data_model.apply(lambda x: get_gender(x.id_card_num), axis = 1)
data_model2['gender'] = data_model2['id_card_num'].apply(get_gender)
data_model2['age'] = data_model2.apply(lambda row: calculate_age1(row['id_card_num'], row['下单日期']), axis=1)

data_model2[:2]

data_model3 = pd.concat([data_model2, data_model1])
data_model3.to_excel('F:/需求/总需求/性别年龄.xlsx', index=False)

# 拒绝理由

In [ ]:
df_reject = pd.read_excel('F:/租后表分析数据_ld/riskdemo2.xlsx')
df_rejected = df_reject[df_reject.强拒结果==1]
# 前置风控
df_rejected_qz = df_rejected[df_rejected.前置风控.notna()]
df_rejected_qz.loc[:, 'rejected'] = df_rejected_qz.前置风控.apply(lambda x: jsonpath(json.loads(x), '$..rejected')[0])
df_rejected_qz = df_rejected_qz[df_rejected_qz.rejected.astype(str)!='[]']
# 前置风控
df_rejected_hz = df_rejected[df_rejected.后置风控.notna()]
df_rejected_hz.loc[:, 'rejected'] = df_rejected_hz.后置风控.apply(lambda x: jsonpath(json.loads(x), '$..rejected')[0])
df_rejected_hz = df_rejected_hz[df_rejected_hz.rejected.astype(str)!='[]']

# 出库前风控
df_rejected_ck = df_rejected[df_rejected.出库前风控.notna()]
df_rejected_ck.loc[:, 'rejected'] = df_rejected_ck.出库前风控.apply(lambda x: jsonpath(json.loads(x), '$..rejected')[0])
df_rejected_ck = df_rejected_ck[df_rejected_ck.rejected.astype(str)!='[]']
df_rejected_ck.rejected.drop_duplicates()
# df_concat = pd.concat([df_rejected_qz[['手机号', 'rejected']], df_rejected_hz[['手机号', 'rejected']], df_rejected_ck[['手机号', 'rejected']]])
# df_concat = df_concat.explode('rejected')
# pd.crosstab('2024-01-21', df_concat['rejected'], margins=True).to_excel('F:/需求/总需求/拒绝理由.xlsx')

In [ ]:

df_rejected_qz.rejected.drop_duplicates()

In [ ]:
df_rejected_hz.rejected.drop_duplicates()

# 仕程需求

In [ ]:
# df_fp = pd.read_excel('F:/无标题(2).xlsx')
# df_fp = df_fp[~df_fp.agent_user_name.isna()]
# df_fp.loc[:, 'sort'] = df_fp.sort.str.strip('[]')
# df_fp.loc[:, '还款'] = np.where(df_fp.status==3, 1, 0)
# df_fp.loc[:, '未还款'] = np.where(df_fp.status!=3, 1, 0)
# df_fp_g = df_fp.groupby(['agent_user_name', 'sort']).agg({'order_id': 'size', '还款': 'sum', '未还款': 'sum'}).reset_index()
# df_fp_g = df_fp_g.rename(columns={'agent_user_name': '名称', 'sort': '期数', 'order_id': '总任务'})
# df_fp_g.loc[:, '催回率'] = (df_fp_g.还款/df_fp_g.总任务).map(lambda x: format(x, '.2%'))
# # with pd.ExcelWriter('F:/需求/总需求/还款情况.xlsx', engine='xlsxwriter') as writer:
#     # df_fp_g.to_excel(writer, sheet_name='还款情况', index=False)


In [ ]:
df_6month = pd.read_excel('D:/需求/仕程需求/6月起失联名单.xlsx')
df_6month


In [ ]:
order_list = tuple(df_6month.订单号.to_list())
sql_6month = f'''
select 
om.order_number, om.status, ymos.refund_date, ymos.reality_refund_date, ymos.sort,
tprm.rembursement_status
from db_digua_business.t_order om
left join db_rent.ya_merchant_order_stages ymos on om.id = ymos.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on om.id = tprm.order_id
where om.order_number in {order_list}
'''
df_month = query(sql_6month)
now_day = datetime.now().strftime('%Y-%m-%d')
# 筛选到今天为止都没有还款或归还逾期(逾期且待归还)的订单
df_month = df_month[((df_month.refund_date<now_day)&(df_month.reality_refund_date.isna()))|((df_month.rembursement_status=="已逾期")&(df_month.status==5))]
# 基于订单号去重
df_month.drop_duplicates(subset=['order_number'], keep='first', inplace=True)
df_month

In [ ]:
month_list = df_month.order_number.to_list()
df_6month.loc[:, '是否逾期'] = np.where(df_6month.订单号.isin(month_list), '是', '否')
df_6month = df_6month.merge(df_month[['sort', 'order_number']],left_on='订单号' ,right_on = 'order_number', how = 'left')


In [ ]:
df_6month.to_excel('D:/需求/仕程需求/6月起失联名单_新.xlsx', index=False)

# 力哥

In [ ]:
# df  = pd.read_excel('F:/需求/力哥需求/4月份呼入.xlsx', sheet_name='Sheet1')
# df.loc[:, '分钟'] = df.通话时长.str[3:5]
# df.loc[:, '秒'] = df.通话时长.str[-2:]
# df.to_excel('F:/需求/力哥需求/4月份呼入_新.xlsx', index=False)

# 资方数据

In [ ]:
sql_zf = '''
SELECT tprm.order_number, om.merchant_name,tod.sku_attributes, tso.price as 碎屏险价格,
tod.new_actual_money '应收金额', tprm.purchase_amount '采购成本', (tprm.rent_received) '已收租金', tprm.overdue_type, top.residue_money '押金'
,case when ym.`name`='维客自营账户' then '维客自营账户' 
			when ym.`name`='海南鸿丰商业保理有限公司' then '海南鸿丰商业保理有限公司'
			when ym.`name`='成都越蓉金科科技有限公司' then '成都越蓉金科科技有限公司'
			when ym.`name`='杭州金锤网络信息服务有限公司' then '杭州金锤网络信息服务有限公司'
			else '其他' end '资方'
FROM db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join (SELECT order_id, residue_money from db_digua_business.t_order_pledge where residue_money>=0) top on top.order_id=om.id
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id
where om.`status`=4
'''
df_zf = query(sql_zf)

In [ ]:
# M0：1-30
# M1:31-60
# M2：61-90
# M3:91-180
# M6：>180
df_zf = df_zf[df_zf.采购成本>0]
df_zf = clean.drop_merchant(df_zf)
df_zf = clean.drop_rejected_merchant(df_zf)
df_zf.loc[:, '押金'] = df_zf.押金.fillna(0)
df_zf.loc[:, '已收租金'] = df_zf.已收租金+df_zf.押金
df_zf.loc[:, '应收未收金额'] = df_zf.应收金额-df_zf.已收租金
df_zf.loc[:, '应收未收采购成本'] = df_zf.采购成本-df_zf.已收租金
df_zf.loc[:, '应收未收中未逾期的金额'] = np.where(df_zf.overdue_type=='', df_zf.应收未收金额, 0)
df_zf.loc[:, '应收未收中DPD1-30的金额'] = np.where(df_zf.overdue_type=='M0', df_zf.应收未收金额, 0)
df_zf.loc[:, '应收未收中DPD31-60的金额'] = np.where(df_zf.overdue_type=='M1', df_zf.应收未收金额, 0)
df_zf.loc[:, '应收未收中DPD61+的金额'] = np.where(df_zf.overdue_type.isin(['M2', 'M3', 'M4', 'M5', 'M6']), df_zf.应收未收金额, 0)
df_zf.loc[:, '应收未收中未逾期的采购成本'] = np.where(df_zf.overdue_type=='', df_zf.应收未收采购成本, 0)
df_zf.loc[:, '应收未收中DPD1-30的采购成本'] = np.where(df_zf.overdue_type=='M0', df_zf.应收未收采购成本, 0)
df_zf.loc[:, '应收未收中DPD31-60的采购成本'] = np.where(df_zf.overdue_type=='M1', df_zf.应收未收采购成本, 0)
df_zf.loc[:, '应收未收中DPD61+的采购成本'] = np.where(df_zf.overdue_type.isin(['M2', 'M3', 'M4', 'M5', 'M6']), df_zf.应收未收采购成本, 0)
df_zf_group = df_zf.groupby('资方').agg({'应收金额': 'sum', '应收未收金额': 'sum', '应收未收中未逾期的金额': 'sum', '应收未收中DPD1-30的金额': 'sum', '应收未收中DPD31-60的金额': 'sum', 
                            '应收未收中DPD61+的金额': 'sum', '采购成本': 'sum', '应收未收采购成本': 'sum', '应收未收中未逾期的采购成本': 'sum', '应收未收中DPD1-30的采购成本' : 'sum',
                            '应收未收中DPD31-60的采购成本': 'sum', '应收未收中DPD61+的采购成本': 'sum', '碎屏险价格': 'sum'})
df_zf_group.sort_values('资方', ascending=False).to_excel('D:/需求/总需求/资方数据.xlsx')

In [ ]:
df_zf[(df_zf.应收未收中未逾期的采购成本 >= 0)&(df_zf.资方 == '维客自营账户')].应收未收中未逾期的采购成本.sum()
df_zf[(df_zf.应收未收中未逾期的采购成本 < 0)&(df_zf.资方 == '维客自营账户')].应收未收中未逾期的采购成本.sum()

In [ ]:
sql_zf2 = '''
select
om.id, om.status, ym.name, om.create_time
from db_digua_business.t_order om
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
where ym.name in ('海南鸿丰商业保理有限公司', '杭州金锤网络信息服务有限公司')
'''
df_zf2 = query(sql_zf2)

In [ ]:
df_zf2.loc[:, '下单月份'] = df_zf2.create_time.dt.strftime('%Y-%m')
df_zf2.loc[:, '进件'] = np.where(df_zf2.status.isin([1,13]), 0, 1)
df_zf2.loc[:, '出库'] = np.where(df_zf2.status.isin([2,3,4,5,6,8,15]), 1, 0)
df_zf2_group = df_zf2.groupby('下单月份').agg({'进件': 'sum', '出库': 'sum'})
df_zf2_group.loc['合计', :] = df_zf2_group.sum(axis=0)
df_zf2_group#.to_excel('F:/需求/七月需求/资方数据_金锤和鸿丰.xlsx')

# 财务数据

In [ ]:
def cw(month):
    sql_cw = f'''
    SELECT
    om.order_number 订单号, ym.name 名称, ymos.sort 期数, ymos.money 租金
    from db_digua_business.t_order om 
    left join db_rent.ya_merchant ym on om.merchant_id=ym.shop_id
    left join db_rent.ya_merchant_order_stages ymos on ymos.order_id=om.id
    where om.status=4
    and ym.shop_id in (14, 36, 148)
    and date_format(ymos.refund_date, '%Y-%m')='{month}'
    '''
    df_cw = query(sql_cw)
    df_cw_g = df_cw.groupby('名称').agg({'租金': 'sum'}).rename(columns={'租金': '应收租金'}).rename_axis('归属')
    df_cw_g.loc['合计',:] = df_cw_g.应收租金.sum()
    return df_cw, df_cw_g

# df_cw.to_excel('F:/需求/总需求/财务数据.xlsx', index=False)

In [ ]:
month = 5
# df_cw1, df_cw1_g = cw('2025-01')
# df_cw2, df_cw2_g = cw('2025-02')
# df_cw3, df_cw3_g = cw('2025-03')
# TO_DO财务数据改时间
df_cw5, df_cw5_g = cw('2025-05')
with pd.ExcelWriter(f'D:/需求/总需求/财务数据_{month}月.xlsx', engine='xlsxwriter') as writer:
    # df_cw1.to_excel(writer, sheet_name='一月份明细', index=False)
    # df_cw1_g.to_excel(writer, sheet_name='一月份汇总')
    # df_cw2.to_excel(writer, sheet_name='二月份明细', index=False)
    # df_cw2_g.to_excel(writer, sheet_name='二月份汇总')
    # df_cw3.to_excel(writer, sheet_name='三月份明细', index=False)
    # df_cw3_g.to_excel(writer, sheet_name='三月份汇总')
    df_cw5.to_excel(writer, sheet_name='五月份明细', index=False)
    df_cw5_g.to_excel(writer, sheet_name='五月份汇总')

In [ ]:
sql_qg = '''
select 
tprm.relet_periods,tod.new_actual_money,tprm.rent_received,
tprm.overdue_periods,tprm.total_receivable, tm.name
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_merchant tm on tm.id=om.merchant_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
where tm.`name` in ('南宁微美汇', '广州捷瑞环保科技')
'''
df_qg = query(sql_qg)


In [ ]:
# relet_periods续租期数，total_receivable总应收，rent_received已收租金，new_actual_money新买断
df_qg.loc[:, '续租剩余总未收'] = np.where(df_qg.relet_periods>0, df_qg.new_actual_money-df_qg.rent_received, 0)
df_qg.loc[:, '逾期总应收'] = np.where(df_qg.overdue_periods>0, df_qg.total_receivable, 0)
df_qg.groupby('name').agg({'续租剩余总未收': 'sum', '逾期总应收': 'sum'}).to_excel('D:/需求/奇哥需求/商家数据.xlsx')

# 廖志彧需求

In [ ]:
# 按结束时间买断订单数，ya_buyout_task促买断表
sql_lzy = '''
select
ybt.order_id 订单ID, tod.new_actual_money 买断价, too.discount_money 优惠金额, (tod.new_actual_money-too.discount_money) 实收买断金额,
tprm.purchase_amount 采购金额, ybt.`status`, ybt.follow_log_tag, ybt.type, om.has_actual, date_format(ybt.end_time, '%Y-%m') 月份
from db_rent.ya_buyout_task ybt
left join (select order_id, discount_money from db_digua_business.t_order_out where status=2) too on too.order_id=ybt.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=too.order_id
left join db_digua_business.t_order_details tod on tod.order_id=too.order_id
left join db_digua_business.t_order om on om.id=ybt.order_id
where date_format(ybt.end_time, '%Y-%m')='2025-05'
'''
# 提前买断订单对应期数
sql_stages = '''
select order_id 订单ID, sort 期数 from db_rent.ya_merchant_order_stages where date_format(refund_date, '%Y-%m')='2025-05'
'''
# 按发布时间买断订单数
sql_amd = '''
select ybt.order_id 发布时间订单号, tod.new_actual_money 发布时间买断价, date_format(ybt.add_time, '%Y-%m') 月份 from db_rent.ya_buyout_task ybt 
left join db_digua_business.t_order_details tod on tod.order_id=ybt.order_id
where date_format(ybt.add_time, '%Y-%m')='2025-05'
'''
df_amd = query(sql_amd)

df_amd_g = df_amd.groupby('月份').agg({'发布时间订单号': 'count', '发布时间买断价': 'sum'})
df_lzy = query(sql_lzy)
df_stages = query(sql_stages)

# 发布买断订单数、买断订单数、发布买断金额（采购金额）、发布买断金额（买断价）、实收买断金额、实际买断订单数、买断优惠券总额、发放优惠前的毛利率、发放优惠后的毛利率

In [ ]:
df_lzy.loc[:, '实际买断订单数'] = np.where((df_lzy.status.isin([3, 4]))&(df_lzy.has_actual==1), 1, 0)
df_lzy.loc[:, '实收买断金额'] = np.where((df_lzy.status.isin([3, 4]))&(df_lzy.has_actual==1), df_lzy.实收买断金额, 0)
def md(df):
    df_g = df.groupby('月份').agg({'订单ID': 'count', '采购金额': 'sum', '买断价': 'sum', '实际买断订单数': 'sum', '实收买断金额': 'sum', '优惠金额': 'sum'}).rename(columns={'订单ID': '发布买断订单数'})
    df_g.loc[:, '优惠比例'] = (df_g.优惠金额/df_g.采购金额).map(lambda x: format(x, '.2%'))
    df_g.loc[:, '发放优惠前的毛利率'] = ((df_g.买断价-df_g.采购金额)/df_g.采购金额).map(lambda x: format(x, '.2%'))
    df_g.loc[:, '发放优惠后的毛利率'] = ((df_g.实收买断金额-df_g.采购金额)/df_g.采购金额).map(lambda x: format(x, '.2%'))
    return df_g
df_lzy_ts = md(df_lzy[(df_lzy.status.isin([3, 4]))&(df_lzy.follow_log_tag==4)&(df_lzy.has_actual==1)])
df_lzy_tq = df_lzy[(df_lzy.status.isin([3, 4]))&(df_lzy.follow_log_tag!=4)&(df_lzy.type==1)&(df_lzy.has_actual==1)]
df_lzy_tq2 = md(df_lzy_tq)
df_lzy_tq3 = df_lzy_tq.merge(df_stages, on='订单ID', how='left')
df_lzy_dq = df_lzy[(df_lzy.status.isin([3, 4]))&(df_lzy.follow_log_tag!=4)&(df_lzy.type==0)&(df_lzy.has_actual==1)]
# df_lzy_dq = md(df_lzy_dq)
# df_lzy_dq2 = df_lzy_dq.merge(df_amd_g, on='月份', how='left')
# with pd.ExcelWriter('F:/需求/廖志彧需求/5月买断订单.xlsx', engine='xlsxwriter') as writer:
#     df_lzy_ts.to_excel(writer, sheet_name='特殊申请')
#     df_lzy_tq2.to_excel(writer, sheet_name='提前买断')
#     df_lzy_tq3.to_excel(writer, sheet_name='提前买断明细', index=False)
#     df_lzy_dq.to_excel(writer, sheet_name='到期买断')
# df_lzy.to_excel('F:/需求/廖志彧需求/5月买断订单.xlsx', index=False)&(df_lzy.has_actual==1)

In [ ]:
sql_yz = '''
select
om.id, turt.discount_status, tdi.money, tdi.type, tprm.all_rental, tprm.all_deposit, tprm.purchase_amount, om.status, tod.new_actual_money, date_format(om.create_time, '%Y-%m') 月份
from db_digua_business.t_user_recall_task turt 
left join db_digua_business.t_member_user tmu on tmu.id_card_num=turt.id_card_num
left join db_digua_business.t_order om on om.user_id=tmu.id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_discount_item tdi on tdi.user_id=om.user_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
where date_format(om.create_time, '%Y-%m')='2025-05'
'''
df_yz = query(sql_yz)
df_yz = df_yz.drop_duplicates(subset=['id'])
df_yz.loc[:, '进件订单数'] = np.where((df_yz.status==1)|(df_yz.status==13), 0, 1)
df_yz.loc[:, '出库订单数'] = np.where(df_yz.status.isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
df_yz.loc[:, '优惠券金额'] = np.where((df_yz.discount_status=='Y')&(df_yz.type==1), df_yz.money, 0)
df_yz.loc[:, '买断价'] = np.where(df_yz.出库订单数==1, df_yz.new_actual_money, 0)
df_yz_g = df_yz.groupby('月份').agg({'id': 'count', '进件订单数': 'sum', '出库订单数': 'sum', '优惠券金额': 'sum', 'purchase_amount': 'sum', 'all_rental': 'mean', 'all_deposit': 'mean', '买断价': 'sum'}).rename(columns={'id': '发布优召订单数', 'purchase_amount': '采购成本', 'all_rental': '总租金均值', 'all_deposit': '总押金均值'})
df_yz_g.loc[:, '租售比'] = (df_yz_g.总租金均值/df_yz_g.总押金均值).map(lambda x: format(x, '.2%'))
df_yz_g.loc[:, '优惠比例'] = (df_yz_g.优惠券金额/df_yz_g.采购成本).map(lambda x: format(x, '.2%'))
df_yz_g.loc[:, '优惠前毛利率'] = ((df_yz_g.买断价-df_yz_g.采购成本)/df_yz_g.采购成本).map(lambda x: format(x, '.2%'))
df_yz_g.loc[:, '优惠后毛利率'] = ((df_yz_g.买断价-df_yz_g.采购成本-df_yz_g.优惠券金额)/df_yz_g.采购成本).map(lambda x: format(x, '.2%'))
# 发布优召订单数、进件订单数、出库订单数、优惠券金额、采购成本、租售比
with pd.ExcelWriter('D:/需求/廖志彧需求/5月优召订单.xlsx', engine='xlsxwriter') as writer:
    df_yz_g.to_excel(writer, sheet_name='优召')

In [ ]:
sql_wl = '''
select  
tort.order_id, tprm.purchase_amount, tod.new_actual_money, tdi.money, tdi.type, om.status, date_format(om.create_time, '%Y-%m') 月份, tort.task_status
from db_digua_business.t_order_retention_task tort
left join db_digua_business.t_order om on om.id=tort.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_discount_item tdi on tdi.user_id=om.user_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
where date_format(om.create_time, '%Y-%m')='2025-05'
'''
df_wl = query(sql_wl)
df_wl.loc[:, '挽留成功订单数'] = np.where(df_wl.task_status=='SUCCESS', 1, 0)
df_wl.loc[:, '出库'] = np.where(df_wl.status.isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
df_wl.loc[:, '买断优惠券'] = np.where(df_wl.type==4, df_wl.money, 0)
df_wl.loc[:, '租中优惠券'] = np.where(df_wl.type==1, df_wl.money, 0)
df_wl.loc[:, '优惠券总金额'] = np.where(df_wl.type.isin([1, 4]), df_wl.money, 0)
df_wl.loc[:, '采购金额'] = np.where(df_wl.status.isin([2, 3, 4, 5, 6, 8, 15]), df_wl.purchase_amount, 0)
df_wl.loc[:, '买断价'] = np.where(df_wl.status.isin([2, 3, 4, 5, 6, 8, 15]), df_wl.new_actual_money, 0)
df_wl_g = df_wl.groupby('月份').agg({'order_id': 'count', '挽留成功订单数': 'sum', '出库': 'sum', '采购金额': 'sum', '买断价': 'sum', '优惠券总金额': 'sum', '买断优惠券': 'sum', '租中优惠券': 'sum'}).rename(columns={'order_id': '挽留发布订单数'})
df_wl_g.loc[:, '优惠比例'] = (df_wl_g.优惠券总金额/df_wl_g.采购金额).map(lambda x: format(x, '.2%'))
df_wl_g.loc[:, '买断优惠券比例'] = (df_wl_g.买断优惠券/df_wl_g.采购金额).map(lambda x: format(x, '.2%'))
df_wl_g.loc[:, '租中优惠券比例'] = (df_wl_g.租中优惠券/df_wl_g.采购金额).map(lambda x: format(x, '.2%'))
df_wl_g.loc[:, '发放优惠前的毛利率'] = ((df_wl_g.买断价-df_wl_g.采购金额)/df_wl_g.采购金额).map(lambda x: format(x, '.2%'))
df_wl_g.loc[:, '发放优惠后的毛利率'] = ((df_wl_g.买断价-df_wl_g.优惠券总金额-df_wl_g.采购金额)/df_wl_g.采购金额).map(lambda x: format(x, '.2%'))
df_wl_g
# 优惠券（买断优惠券、租中优惠券）总金额、挽留发布任务数、采购金额、买断价、优惠比例（优惠券/采购金额）、发放优惠前的毛利率、发放优惠后的毛利率
with pd.ExcelWriter('D:/需求/廖志彧需求/5月挽留订单.xlsx', engine='xlsxwriter') as writer:
    df_wl_g.to_excel(writer, sheet_name='挽留')

# 倩姐需求

## 每个月的部分免押和非免押的订单数

In [ ]:
sql = '''
select  om.create_time, om.id, top.total_freeze_fund_amount

from db_digua_business.t_order om
left join db_digua_business.t_order_pay top on om.id=top.order_id
where top.pay_type='ZFBYSQ'
and top.`status` in (2,5)
and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2024-01-01'
and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-07-31 '
'''
df_order = clean.query(sql)



In [ ]:
df = df_order.copy()
df = df[df.type!=4]

